In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://gohighlanders.com/sports/mens-basketball/stats/2024-25?path=mbball'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [4]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [5]:
#ISSUE: COLUMNS ARE OUT OF ORDER
#After #, GP, GS it goes out of order
    #It jumps to 

#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)

[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [6]:
#Extract data for columns
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data

['24', 'Hargress, Barrington', '27', '27', '855', '31.7', '197', '454', '.434', '60', '187', '.321', '81', '111', '.730', '535', '19.8', '9', '49', '58', '2.1', '48', '106', '57', '29', '1', 'View Bio']
['03', 'Moses, Isaiah', '27', '13', '689', '25.5', '106', '257', '.412', '42', '123', '.341', '48', '51', '.941', '302', '11.2', '5', '36', '41', '1.5', '53', '77', '45', '19', '1', 'View Bio']
['02', 'Smith, Kaleb', '27', '27', '722', '26.7', '100', '283', '.353', '45', '142', '.317', '50', '68', '.735', '295', '10.9', '37', '88', '125', '4.6', '51', '31', '40', '13', '0', 'View Bio']
['11', 'Pickens, Nate', '26', '25', '717', '27.6', '72', '188', '.383', '37', '92', '.402', '54', '76', '.711', '235', '9.0', '32', '62', '94', '3.6', '68', '50', '42', '21', '12', 'View Bio']
['21', 'Armotrading, Joel', '26', '24', '578', '22.2', '68', '110', '.618', '0', '0', '.000', '31', '44', '.705', '167', '6.4', '66', '110', '176', '6.8', '76', '16', '25', '11', '31', 'View Bio']
['10', 'Olabode, N

In [7]:
df

#Export this into a csv
df.to_csv(r'/Users/ryanuyeki/Downloads/asa/ucr_player_stats.csv', index=False, encoding = "utf-8")